<a href="https://colab.research.google.com/github/JabborovRoboCoder/Airfare-price-prediction/blob/main/Aviachipta_narxini_bashorat_qilish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import  ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

In [ ]:
!unzip /content/aviachipta-narxini-bashorat-qilish.zip

Archive:  /content/aviachipta-narxini-bashorat-qilish.zip
  inflating: sample_solution.csv     
  inflating: test_data.csv           
  inflating: train_data.csv          


In [ ]:
df_tr = pd.read_csv("/content/train_data.csv")
df_ts = pd.read_csv("/content/test_data.csv")
s_s = pd.read_csv("/content/sample_solution.csv")

In [ ]:
df_tr.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [ ]:
print(df_tr.info())
print(df_ts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------ 

In [ ]:
print(df_tr.shape)
print(df_ts.shape)

(20000, 12)
(5000, 11)


In [ ]:
print(df_tr.isnull().sum())
print(df_ts.isnull().sum())

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64
id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
dtype: int64


In [ ]:
df_tr['days_left'].unique()

array([21,  7,  5, 28,  4, 20, 30, 12, 18, 22, 46,  3, 25, 40, 34, 15, 23,
       17, 13, 41, 33, 43,  8,  6, 26, 47, 36, 11, 19, 14, 35,  9, 32, 37,
       44, 27, 31, 38, 42, 16, 45, 10, 24, 49,  2, 48, 29, 39,  1])

In [ ]:
# encoder = LabelEncoder()
# df_tr['airline'] = encoder.fit_transform(df_tr['airline'].values)
# df_tr['class'] = encoder.fit_transform(df_tr['class'].values)
# df_tr['arrival_time'] = encoder.fit_transform(df_tr['arrival_time'].values)
# df_tr['departure_time'] = encoder.fit_transform(df_tr['departure_time'].values)
# df_tr['source_city'] = encoder.fit_transform(df_tr['source_city'].values)
# df_tr['stops'] = encoder.fit_transform(df_tr['stops'].values)
# df_tr['destination_city'] = encoder.fit_transform(df_tr['destination_city'].values)
# df_tr['flight'] = encoder.fit_transform(df_tr['flight'].values)

# df_tr.sample(10)

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
6641,6642,5,1302,2,5,0,4,0,1,12.92,8,11100
6681,6682,1,736,4,4,0,0,3,1,24.83,44,4799
18861,18862,1,608,1,2,2,2,2,1,2.83,41,2271
19187,19188,5,1194,4,1,0,2,5,1,11.17,17,13052
10026,10027,3,156,4,4,0,2,3,1,8.58,31,3401
1286,1287,5,1208,4,0,0,1,5,1,16.58,13,11675
12418,12419,5,1242,2,0,0,5,5,0,9.08,32,51384
17784,17785,3,495,1,2,0,5,3,1,6.75,43,1105
9996,9997,1,658,5,4,0,5,2,1,14.17,31,3843
18418,18419,1,591,2,0,0,1,3,0,18.33,30,44405


In [ ]:
# df_tr.corrwith(df_tr['price']).abs().sort_values(ascending=False)

In [ ]:
train_set,test_set = train_test_split(df_tr,test_size=0.1,random_state=42)

In [ ]:
x_num = train_set[['duration','days_left']]
x_train =train_set.drop(["price","flight"], axis=1)
y = train_set["price"].copy()

num_pipeline = Pipeline([
    ('std_scaler',StandardScaler()),
    ('imputer',SimpleImputer(strategy='median'))
])

num_attribs = list(x_num)
cat_attribs = ['airline','class','arrival_time','departure_time','source_city','stops','destination_city']

full_pipeline = ColumnTransformer([
    ('num',num_pipeline,num_attribs),
    ('cat',OneHotEncoder(),cat_attribs)
])

x_prepared =full_pipeline.fit_transform(x_train)

In [ ]:
#Start Learning

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_prepared,y)

LinearRegression()

In [ ]:
test_data = x_train.sample(10)

test_label = y.loc[test_data.index]

In [ ]:
test_data_prepared = full_pipeline.transform(test_data)


In [ ]:
predicted_data = lr_model.predict(test_data_prepared)

pd.DataFrame({'Prognoz':predicted_data,"Real_baxosi":test_label})

,Prognoz,Real_baxosi
9549,53180.188478,58125
9301,1058.561598,3862
2511,9671.337911,6067
9862,10726.185157,14396
3645,56786.739733,74273
7486,48968.022825,39547
7256,52433.237347,51707
18517,7779.851967,3462
8888,6276.320729,4359
3992,10400.134161,13758


In [ ]:
x_test = test_set.drop(["price","flight"],axis=1)
y_test = test_set['price'].copy()

In [ ]:
x_test_prepared = full_pipeline.transform(x_test)
y_predicted = lr_model.predict(x_test_prepared)

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_test,y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

6616.747215743838


In [ ]:
#Decion Tree

tree_model= DecisionTreeRegressor()
tree_model.fit(x_prepared,y)

DecisionTreeRegressor()

In [ ]:
y_predicted = tree_model.predict(x_test_prepared)

In [ ]:
lin_mse = mean_squared_error(y_test,y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

4714.113759300363


In [ ]:
# RandomForest Algoritmi

rf_model = RandomForestRegressor()
rf_model.fit(x_prepared,y)

RandomForestRegressor()

In [ ]:
# RMSE
y_predicted = rf_model.predict(x_test_prepared)
lin_mse = mean_squared_error(y_test,y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

3728.0030812979967


In [ ]:
#GBC

gbc_model = GradientBoostingRegressor()
gbc_model.fit(x_prepared,y)

GradientBoostingRegressor()

In [ ]:
# RMSE
y_predicted = rf_model.predict(x_test_prepared)
lin_mse = mean_squared_error(y_test,y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

3728.0030812979967


In [ ]:
gbc_model.score(x_prepared,y)

0.9546016384812822

In [ ]:
df_ts = df_ts.drop("flight",axis=1)

x_test_prepared = full_pipeline.transform(df_ts)

In [ ]:
pred_test_data = gbc_model.predict(x_test_prepared)

abc=pd.DataFrame({'id':s_s.id,'price':pred_test_data})
abc.head()

,id,price
0,1,57499.893560
1,2,56200.663573
2,3,23559.281365
3,4,2792.862555
4,5,6565.335096
